In [1]:
using StatisticalRethinking, CmdStan
gr(size=(600,600));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchains);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:36
└ @ Core string:36
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:36
└ @ Core string:36
Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.033, 0.033, 0.032, 0.034) seconds, 0.13 seconds total
Sampling took (0.050, 0.047, 0.050, 0.048) seconds, 0.20 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -891  2.6e-02     1.0  -893  -891  -891  1.5e+03  7.8e+03  1.0e+00
accept_stat__   0.92  1.4e-03   0.099  0.71  0.96   1.0  4.8e+03  2.5e+04  1.0e+00
stepsize__      0.77  4.1e-02   0.058 

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters │ mean    │ std      │ naive_se   │ mcse       │ ess     │
│     │ Symbol     │ Float64 │ Float64  │ Float64    │ Float64    │ Any     │
├─────┼────────────┼─────────┼──────────┼────────────┼────────────┼─────────┤
│ 1   │ mu         │ 154.593 │ 0.399915 │ 0.00632321 │ 0.0072299  │ 3578.23 │
│ 2   │ sigma      │ 7.76996 │ 0.300933 │ 0.00475817 │ 0.00714609 │ 2620.12 │

Quantiles

│ Row │ parameters │ 2.5%    │ 25.0%   │ 50.0%   │ 75.0%   │ 97.5%   │
│     │ Symbol     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ mu         │ 153.805 │ 154.316 │ 154.598 │ 154.866 │ 155.377 │
│ 2   │ sigma      │ 7.21036 │ 7.56407 │ 7.76015 │ 7.9621  │ 8.40414 │


end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*